<a href="https://colab.research.google.com/github/Abdellahbado/ResNet50-Malaria-Classification/blob/main/resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!mkdir -p ~/.kaggle
!cp /content/drive/My\ Drive/Kaggle\ API/kaggle.json ~/.kaggle/


In [4]:
!chmod 600 ~/.kaggle/kaggle.json
!pip install kaggle


In [5]:
!kaggle datasets download -d iarunava/cell-images-for-detecting-malaria -p /content/drive/My\ Drive/MalariaDataset --force


 99% 671M/675M [00:12<00:00, 46.7MB/s]
100% 675M/675M [00:12<00:00, 54.6MB/s]


In [ ]:
import zipfile
import os

zip_file_path = '/content/drive/My Drive/MalariaDataset/cell-images-for-detecting-malaria.zip'

extracted_dir_path = '/content/drive/My Drive/MalariaDataset'

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir_path)

extracted_files = os.listdir(extracted_dir_path)
print("Files extracted successfully:", extracted_files)


In [ ]:
directory = "cell_images/"

# Create a dataset from the directory
dataset = tf.keras.utils.image_dataset_from_directory(
    directory,
    labels="inferred",
    shuffle=True,
    label_mode="int",
    color_mode='rgb',
    batch_size=32
)
dataset

Found 27558 files belonging to 2 classes.


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224))
    return image, label

In [ ]:
# Calculate the number of samples in the training dataset

num_train_samples = len(dataset)

In [ ]:
 # Split the dataset further into training and validation sets
train_dataset = dataset.take(int(0.7 * num_train_samples))
val_dataset = dataset.skip(int(0.7 * num_train_samples))

num_valid = len(val_dataset)

val_dataset = val_dataset.take(int(0.5 * num_valid))
test_dataset = val_dataset.skip(int(0.5 * num_valid))

In [ ]:
train_dataset = train_dataset.map(preprocess_image)
val_dataset = val_dataset.map(preprocess_image)
test_dataset = test_dataset.map(preprocess_image)

train_dataset

<_MapDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
input_shape = (224, 224, 3)
num_classes = 2

base_model = keras.applications.resnet50.ResNet50(
    weights=None, include_top=False, input_shape=input_shape
)

x = keras.layers.GlobalAveragePooling2D()(base_model.output)
x = keras.layers.Dense(128, activation="relu")(x)
predictions = keras.layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
epochs = 10
batch_size = 32

history = model.fit(train_dataset,epochs=epochs,batch_size=batch_size,validation_data=val_dataset)

Epoch 1/10
 23/603 ━━━━━━━━━━━━━━━━━━━━ 1:19:19 8s/step - accuracy: 0.6172 - loss: 1.2642

In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset)

# Print the test loss and accuracy
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)